<a href="https://colab.research.google.com/github/cindyhps/google_collab/blob/main/adversarial_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Library

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tarfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path ke file tar.gz di Google Drive
tar_file_paths = [
    '/content/drive/My Drive/adversarial/part1.tar.gz',
    '/content/drive/My Drive/adversarial/part2.tar.gz',
    '/content/drive/My Drive/adversarial/part3.tar.gz',
]

# Direktori tujuan untuk mengekstrak file
extract_to = '/content/extracted_data'

# Buat direktori tujuan jika belum ada
os.makedirs(extract_to, exist_ok=True)

# Ekstrak file dari setiap file tar.gz
for tar_file_path in tar_file_paths:
    with tarfile.open(tar_file_path, 'r:gz') as tar:
        tar.extractall(extract_to)

print(f"File telah diekstrak ke {extract_to}")

File telah diekstrak ke /content/extracted_data


In [ ]:
# Path ke folder dataset UTKFace
dataset_path = '/content/extracted_data'

# Ambil semua nama file gambar dalam dataset
#image_files = os.listdir(dataset_path)
image_files = [f for f in os.listdir(dataset_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
def extract_labels(filename):
    parts = filename.split('_')
    try:
        age = int(parts[0])
        gender = int(parts[1])
        race = int(parts[2])
    except IndexError:
        print(f"Error saat memproses file: {filename}")
        return None, None, None  # Atau nilai default lainnya

    return age, gender, race

# Tampilkan konten label dan contoh isi
print("Konten Label:")
print("Age, Gender, Race")

Konten Label:
Age, Gender, Race


In [ ]:
# Prepare data and labels
images = []
labels = []
for image_file in image_files:
    age, gender, race = extract_labels(image_file)
    images.append(os.path.join(dataset_path, image_file))
    labels.append([age, gender, race]) # store labels as a list

In [ ]:
# Split data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Print the size of each set
print(f"Jumlah data training: {len(train_images)}")
print(f"Jumlah data testing: {len(test_images)}")

In [ ]:
# Display 5 examples from training set
print("\nContoh data training:")
for i in range(5):
    print(f"Contoh {i+1}:")
    print(f"  Nama File: {train_images[i]}")
    print(f"  Age: {train_labels[i][0]}")
    print(f"  Gender: {train_labels[i][1]}")  # 0: Male, 1: Female
    print(f"  Race: {train_labels[i][2]}")    # 0: White, 1: Black, 2: Asian, 3: Indian, 4: Others

In [ ]:
def load_images_from_folder(folder, img_size=(128, 128)):
    images, labels = [], []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize gambar
                img = img / 255.0  # Normalisasi ke [0, 1]
                images.append(img)
                # Ambil label dari nama file atau file metadata (contoh: Asian_0.jpg)
                label = int(filename.split('_')[1])  # Sesuaikan format filename dataset
                labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Path dataset (ganti sesuai direktori dataset kamu)
train_folder = "path_to_dataset/train"
test_folder = "path_to_dataset/test"

# Load dataset
x_train, y_train = load_images_from_folder(train_folder)
x_test, y_test = load_images_from_folder(test_folder)

print(f"Train data shape: {x_train.shape}, Labels: {y_train.shape}")
print(f"Test data shape: {x_test.shape}, Labels: {y_test.shape}")

Transfer Learning MobileNetV2

In [ ]:
# Load pretrained MobileNetV2 tanpa fully connected layer (include_top=False)
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [ ]:
# Tambahkan classifier custom
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)  # Output untuk 3 kelas (Asian, African, Caucasian)

In [ ]:
# Bangun model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze layer awal MobileNetV2 agar tidak dilatih ulang
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
print("Training model...")
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

In [ ]:
# Simpan model setelah training
model.save("mobilenetv2_face_classifier.h5")

FSGM (Fast Gradient Sign Method)

In [ ]:
# Fungsi untuk FGSM Attack
def fgsm_attack(image, label, model, epsilon):
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    label = tf.convert_to_tensor(label, dtype=tf.int64)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.sparse_categorical_crossentropy(label, prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = image + epsilon * tf.sign(gradient)  # Tambahkan noise
    adversarial_image = tf.clip_by_value(adversarial_image, 0, 1)  # Jaga dalam range valid [0,1]
    return adversarial_image

In [ ]:
# Ambil satu sample gambar dan label
sample_image = x_test[0:1]
sample_label = y_test[0:1]

In [ ]:
# FGSM Attack
epsilon = 0.01  # Besar noise
adversarial_image = fgsm_attack(sample_image, sample_label, model, epsilon)

In [ ]:
# Prediksi pada gambar asli dan adversarial
original_pred = tf.argmax(model(sample_image), axis=1).numpy()[0]
adv_pred = tf.argmax(model(adversarial_image), axis=1).numpy()[0]

Visualisasi Hasil Serangan Adversarial

In [ ]:
# Plot gambar asli dan adversarial
plt.figure(figsize=(8, 4))

In [ ]:
# Gambar asli
plt.subplot(1, 2, 1)
plt.title(f"Original: {original_pred}")
plt.imshow(sample_image[0])
plt.axis('off')


In [ ]:
# Gambar adversarial
plt.subplot(1, 2, 2)
plt.title(f"Adversarial: {adv_pred}")
plt.imshow(adversarial_image[0].numpy())
plt.axis('off')

plt.show()

Evaluasi Model pada Dataset Adversarial

In [ ]:
# Buat adversarial dataset
def create_adversarial_dataset(x_data, y_data, model, epsilon):
    adv_data = []
    for i in range(len(x_data)):
        adv_image = fgsm_attack(x_data[i:i+1], y_data[i:i+1], model, epsilon)
        adv_data.append(adv_image.numpy())
    return np.array(adv_data)

print("Generating adversarial test dataset...")
x_test_adv = create_adversarial_dataset(x_test, y_test, model, epsilon)

In [ ]:
# Evaluasi performa pada dataset asli
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Original Test Accuracy: {acc * 100:.2f}%")

In [ ]:
# Evaluasi performa pada dataset adversarial
loss_adv, acc_adv = model.evaluate(x_test_adv, y_test, verbose=0)
print(f"Adversarial Test Accuracy: {acc_adv * 100:.2f}%")

In [ ]:
print(f"Accuracy Drop due to Adversarial Attack: {acc - acc_adv:.2%}")